# Recognizing CIFAR-10 Images
### imports


In [1]:
from keras.datasets import cifar10 
from keras.utils import np_utils 
from keras.models import Sequential 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D 
from keras.optimizers import SGD, Adam, RMSprop 
import matplotlib.pyplot as plt

Using TensorFlow backend.


### Constants

In [2]:
IMG_CHANNELS = 3
IMG_ROWS = 32 
IMG_COLS = 32
BATCH_SIZE = 128 
NB_EPOCH = 20 
NB_CLASSES = 10 
VERBOSE = 1 
VALIDATION_SPLIT = 0.2 
OPTIM = RMSprop()

### Load Data Set

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() 
print('X_train shape:', X_train.shape) 
print(X_train.shape[0], 'train samples') 
print(X_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 3s 0us/step
X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


### Encoding and Normalization

In [4]:
# convert to categorical 
Y_train = np_utils.to_categorical(y_train, NB_CLASSES) 
Y_test = np_utils.to_categorical(y_test, NB_CLASSES) 

# float and normalization 
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train /= 255 
X_test /= 255

### Build Model

In [5]:
model = Sequential() 
model.add(Conv2D(32, (3, 3), padding='same',input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25))
model.add(Flatten()) 
model.add(Dense(512)) 
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(NB_CLASSES)) 
model.add(Activation('softmax')) 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
activation_2 (Activation)    (None, 512)              

### Train Model

In [6]:
model.compile(loss='categorical_crossentropy', 
              optimizer=OPTIM, 
              metrics=['accuracy']) 
model.fit(X_train, 
          Y_train, 
          batch_size=BATCH_SIZE, 
          epochs=NB_EPOCH, 
          validation_split=VALIDATION_SPLIT, 
          verbose=VERBOSE) 

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 56s 1ms/step - loss: 1.7026 - accuracy: 0.3988 - val_loss: 1.3591 - val_accuracy: 0.5268
Epoch 2/20
40000/40000 [==============================] - 59s 1ms/step - loss: 1.3362 - accuracy: 0.5289 - val_loss: 1.2889 - val_accuracy: 0.5408
Epoch 3/20
40000/40000 [==============================] - 51s 1ms/step - loss: 1.2106 - accuracy: 0.5726 - val_loss: 1.3583 - val_accuracy: 0.5354
Epoch 4/20
40000/40000 [==============================] - 51s 1ms/step - loss: 1.1192 - accuracy: 0.6094 - val_loss: 1.1305 - val_accuracy: 0.6053
Epoch 5/20
40000/40000 [==============================] - 50s 1ms/step - loss: 1.0420 - accuracy: 0.6317 - val_loss: 1.0692 - val_accuracy: 0.6257
Epoch 6/20
40000/40000 [==============================] - 50s 1ms/step - loss: 0.9874 - accuracy: 0.6553 - val_loss: 1.0645 - val_accuracy: 0.6329
Epoch 7/20
40000/40000 [==============================] - 60s 2ms/st

### Validate Model

In [7]:
score = model.evaluate(X_test, 
                       Y_test, 
                       batch_size=BATCH_SIZE, 
                       verbose=VERBOSE) 
print("Test score:", score[0]) 
print('Test accuracy:', score[1])

10000/10000 [==============================] - 5s 472us/step
Test score: 1.0323203575134277
Test accuracy: 0.6801999807357788


### Save the Model

In [8]:
model_json = model.to_json() 

with open('cifar10_architecture.json', 'w') as json_file:
    json_file.write(model_json)
  
model.save_weights('cifar10_weights.h5', overwrite=True)

## Ethics

    The ethical concerns that arise from AI image recognition can be broadly grouped into two categories. One encompassed the gathering and labeling of training data, and the other is the use and proliferation of the trained model. Datasets of training images need to be large. The easiest way to get a large data set is by scraping the internet. Just because images are free to view online does not mean it’s ethical to use them in a training dataset. People often don’t want their faces used in training and artists don’t want their work used either. Without consent and licences is it ethical to use and profit off of other people’s images? Scraping data also leads to biased datasets, and potentially incorrect labeling. The internet is full of falsehoods and training models on this data unfiltered will lead to models that perpetuate these falsehoods. This has occurred in self-driving. Many cars have identification bias that increases the risk of minorities being misidentified and therefore harmed at a higher rate (Kim 2021). 
    Even if a dataset is ethically sourced, once a model is trained it can be used in unethical ways. Identification of persons infringes on people's privacy and is feared will be used to discriminate and target groups. This could be an oppressive government using AI to identify and round up a minority population, or a company using identification to target gambling ads to people who display gambling addiction at a casino (Nature 2020). 
    The ability for machines to identify images has many potential benefits but also ethical issues that can cause damage to our society. We must be careful in our approach to these technologies.



# References

Krizhevsky, A. (2009). Learning Multiple Layers of Features from Tiny Images. https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf

Gulli, A., & Pal, S. (2017). Deep learning with Keras : implement neural networks with Keras on Theano and TensorFlow. Packt Publishing.

Kim, T. (2021, October 11). Opinion: Biased Algorithms Are Training Self-Driving Cars. GovTech. https://www.govtech.com/opinion/opinions-biased-algorithms-are-training-self-driving-cars

Nature. (2020). Facial-recognition research needs an ethical reckoning. Nature, 587(7834), 330–330. https://doi.org/10.1038/d41586-020-03256-7
